In [1]:
pip install xlsxwriter

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\User\venvs\nlp\Scripts\python.exe -m pip install --upgrade pip


In [2]:
!pip install xlsxwriter


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import re
import yaml
from pathlib import Path

# ======================================================
# 0. Beállítások
# ======================================================

# Mappa, ahol a fájlok találhatók
BASE_DIR = Path(".")

# ÚJ szótár fájl neve
DICT_PATH = BASE_DIR / "activity_categories.yaml"

# Elemzendő Excel fájlok listája
EXCEL_FILES = [
    BASE_DIR / "CERV-F3-2024_89db_text_lemmatizalt.xlsx",
    BASE_DIR / "CERV-P-2023_93db_text_lemmatizalt.xlsx",
    BASE_DIR / "F_2023_FK02_259db_text_lemmatizalt.xlsx",
]

# Azonosító oszlop neve az Excelben
ID_COL = "Azonosító"

# ======================================================
# 1. Szótár beolvasása és előkészítése
# ======================================================

with open(DICT_PATH, "r", encoding="utf-8") as f:
    dict_raw = yaml.safe_load(f)

# Ha van "categories" kulcs, onnan vesszük, ha nincs, a gyökér a lista
if isinstance(dict_raw, dict) and "categories" in dict_raw:
    categories_raw = dict_raw["categories"]
else:
    categories_raw = dict_raw

def prepare_categories(categories_raw):
    """
    A YAML szótárból egy feldolgozható struktúrát épít:
    [
      {
        "id": ...,
        "label": ...,
        "patterns": [(term_string, compiled_regex), ...]
      },
      ...
    ]
    """
    cat_info = []

    for cat in categories_raw:
        cid = cat["id"]
        label = cat.get("label", cid)
        terms = cat.get("terms", {})

        patterns = []

        # exact → pontos szavak, szóhatáros kereséssel
        for term in terms.get("exact", []):
            pat = r"\b{}\b".format(re.escape(term))
            patterns.append((term, re.compile(pat, flags=re.IGNORECASE)))

        # regex → szabályos kifejezések
        for rx in terms.get("regex", []):
            patterns.append((rx, re.compile(rx, flags=re.IGNORECASE)))

        cat_info.append(
            {
                "id": cid,
                "label": label,
                "patterns": patterns,
            }
        )

    return cat_info

categories = prepare_categories(categories_raw)

LABEL_MAP = {c["id"]: c["label"] for c in categories}
CAT_IDS = [c["id"] for c in categories]

# ======================================================
# 2. Segédfüggvények
# ======================================================

def build_full_text(df, id_col=ID_COL, new_col="full_text"):
    """
    Összefűzi az összes szöveges (object) oszlopot egyetlen 'full_text' mezővé.
    """
    text_cols = [c for c in df.columns if c != id_col and df[c].dtype == "object"]

    df[new_col] = (
        df[text_cols]
        .fillna("")
        .astype(str)
        .agg(" ".join, axis=1)
        .str.lower()
    )
    return df, text_cols

def count_in_text(text, categories):
    """
    Egy szövegben megszámolja:
    - kategóriánként az összes találatot
    - valamint term-szinten is (hosszú formátumhoz)
    """
    cat_counts = {}
    term_rows = []

    for cat in categories:
        cid = cat["id"]
        label = cat["label"]
        total_cat = 0

        for term_str, creg in cat["patterns"]:
            matches = creg.findall(text)
            n = len(matches)
            total_cat += n

            if n > 0:
                term_rows.append({
                    "id": cid,
                    "label": label,
                    "term": term_str,
                    "count": n,
                })

        cat_counts[cid] = total_cat

    return cat_counts, term_rows

def apply_dictionary_to_df(df, categories, text_col="full_text", id_col=ID_COL):
    """
    Lefuttatja a szótárat minden sorra.
    Visszatér:
      - per_row_df: soronkénti kategóriaszámlálások
      - global_counts: kategória szintű összesítés
      - term_counts_long: hosszú formátum (sor, kategória, term, count)
    """
    row_results = []
    long_term_rows = []

    for _, row in df[[id_col, text_col]].iterrows():
        rid = row[id_col]
        text = row[text_col] if isinstance(row[text_col], str) else ""

        cat_counts, term_rows = count_in_text(text, categories)

        row_dict = {id_col: rid}
        row_dict.update(cat_counts)
        row_results.append(row_dict)

        for tr in term_rows:
            tr[id_col] = rid
            long_term_rows.append(tr)

    per_row_df = pd.DataFrame(row_results)

    # biztosítjuk, hogy minden kategória-oszlop szerepeljen
    for cid in CAT_IDS:
        if cid not in per_row_df.columns:
            per_row_df[cid] = 0

    per_row_df = per_row_df[[id_col] + CAT_IDS]
    per_row_df[CAT_IDS] = per_row_df[CAT_IDS].fillna(0).astype(int)

    # globális összesítés
    global_counts = (
        per_row_df[CAT_IDS]
        .sum()
        .reset_index()
        .rename(columns={"index": "id", 0: "count"})
    )
    global_counts["label"] = global_counts["id"].map(LABEL_MAP)
    global_counts = global_counts[["id", "label", "count"]]

    term_counts_long = pd.DataFrame(long_term_rows)
    if not term_counts_long.empty:
        term_counts_long = term_counts_long[[id_col, "id", "label", "term", "count"]]

    return per_row_df, global_counts, term_counts_long

# ======================================================
# 3. Fő futtatás – minden Excel feldolgozása
# ======================================================

for excel_path in EXCEL_FILES:
    print(f"Feldolgozás: {excel_path.name}")

    df = pd.read_excel(excel_path)

    if ID_COL not in df.columns:
        raise ValueError(f"Hiányzik az '{ID_COL}' oszlop a(z) {excel_path.name} fájlból!")

    df[ID_COL] = df[ID_COL].astype(str)

    # teljes szöveg építése
    df, used_text_cols = build_full_text(df, id_col=ID_COL, new_col="full_text")

    # szótár alkalmazása
    per_row_df, global_counts, term_counts_long = apply_dictionary_to_df(
        df, categories, text_col="full_text", id_col=ID_COL
    )

    # új fájlnév
    out_path = excel_path.with_name(excel_path.stem + "_PRO_counts.xlsx")
    print(f"  -> Mentés: {out_path.name}")

    with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
        per_row_df.to_excel(writer, sheet_name="per_sor_kategoria", index=False)
        global_counts.to_excel(writer, sheet_name="global_kategoria", index=False)
        if not term_counts_long.empty:
            term_counts_long.to_excel(writer, sheet_name="term_hosszu", index=False)

    print("Kész.\n")

print("Minden fájl feldolgozva.")


Feldolgozás: CERV-F3-2024_89db_text_lemmatizalt.xlsx
  -> Mentés: CERV-F3-2024_89db_text_lemmatizalt_PRO_counts.xlsx
Kész.

Feldolgozás: CERV-P-2023_93db_text_lemmatizalt.xlsx
  -> Mentés: CERV-P-2023_93db_text_lemmatizalt_PRO_counts.xlsx
Kész.

Feldolgozás: F_2023_FK02_259db_text_lemmatizalt.xlsx
  -> Mentés: F_2023_FK02_259db_text_lemmatizalt_PRO_counts.xlsx
Kész.

Minden fájl feldolgozva.
